In [1]:
import json, glob, os, sys
from itertools import groupby
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append('..')
from imports import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
REVAL_DIR = os.path.expanduser('~/reval_05')
YLIM_AP = (0, 100)
YLIM_APL = (0, 60)
YLIM_F1 = (0, 1)
COLOR_SML = ['#00ff00', '#ffff00', '#ff0000']

In [2]:
files = glob.glob(f'{REVAL_DIR}/*/*/rich_results.json')
results = [load(f) for f in files]
gb = lambda it, key: groupby(sorted(it, key=key), key)

for key, group in gb(results, lambda x: x['model']):
    print(key, end=' ')

R101 R101_C4 R101_DC5 R101_FPN R50 R50_C4 R50_DC5 R50_FPN X101 

In [3]:
sum(int(r['elapsed']) for r in results)

1054712

In [4]:
def flatten(value):
    if 'results' not in results:
        return value
    result = value.copy()
    aps = {k: v for k, v in result['results']['bbox'].items() if '-' not in k}
    result.update(aps)
    del result['results']
    return result
# flatten(results[0])

In [5]:
c_df = pd.DataFrame([flatten(r) for r in results]).sort_values(['model', 'quality'])
c_df.tail()

,quality,model,elapsed,tp,fp,ex,precision,recall,f1,min_score,score_T,AP,AP50,AP75,APs,APm,APl
133,97,X101,2950.533190,29538,69427,11885,0.298469,0.812935,0.436630,0.050000,None,43.125026,63.823199,47.014898,27.212577,46.190293,54.863181
175,98,X101,2949.976151,29537,69465,11845,0.298348,0.812908,0.436496,0.050000,None,43.064660,63.628361,46.943094,26.896694,46.098858,54.857996
103,99,X101,2952.297939,29549,69465,11835,0.298433,0.813238,0.436634,0.050000,None,43.101607,63.681584,47.133904,27.035522,46.123877,54.916508
112,100,X101,2952.063746,29549,69454,11853,0.298466,0.813238,0.436670,0.050001,None,43.070649,63.715295,46.980151,26.992763,46.103360,54.878414
406,101,X101,2673.639724,29546,69608,11862,0.297981,0.813155,0.436139,0.050000,None,43.046948,63.651725,46.888371,27.205004,46.093387,54.890155


In [6]:
for mc in c_df.model.unique():
    ax = c_df[c_df.model == mc][['quality', 'AP', 'AP50', 'AP75']].plot(x='quality', ylim=YLIM_AP)
    ax.set_title(mc)
    save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/01-APx-{mc}")
    plt.close() # prevent show

Figure saved to 2021-03-01-Plot_reval_05_APs/01-APx-R101.png and 2021-03-01-Plot_reval_05_APs/01-APx-R101.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/01-APx-R101_C4.png and 2021-03-01-Plot_reval_05_APs/01-APx-R101_C4.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/01-APx-R101_DC5.png and 2021-03-01-Plot_reval_05_APs/01-APx-R101_DC5.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/01-APx-R101_FPN.png and 2021-03-01-Plot_reval_05_APs/01-APx-R101_FPN.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/01-APx-R50.png and 2021-03-01-Plot_reval_05_APs/01-APx-R50.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/01-APx-R50_C4.png and 2021-03-01-Plot_reval_05_APs/01-APx-R50_C4.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/01-APx-R50_DC5.png and 2021-03-01-Plot_reval_05_APs/01-APx-R50_DC5.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/01-APx-R50_FPN.png and 2021-03-01-Plot_reval_05_APs/01-APx-R50_FPN.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/01-APx-X101.png and 2021-03-01-Plot_rev

In [7]:
avg_pivot = c_df.pivot_table(index='model', values=['AP', 'APl', 'APm', 'APs']).sort_values('AP', ascending=False)
avg_pivot

,AP,APl,APm,APs
model,,,,
X101,34.927685,45.920218,37.778516,20.122565
R101_FPN,34.167798,45.743412,37.028163,19.025153
R101_C4,33.608163,47.307410,37.294725,16.611395
R101_DC5,33.227068,45.751765,36.755736,16.885205
R101,32.810577,43.515592,36.124342,17.436166
R50_FPN,32.528388,43.335347,35.291713,18.184412
R50_DC5,31.721918,43.682657,35.189993,16.106194
R50,31.080935,41.548232,34.032183,16.989777
R50_C4,31.079360,44.221552,34.446147,15.153276


In [8]:
def _save_plot(plot, name='', title=None):
    if title:
        plot.set_title(title)
    fig = plot.get_figure()
    fig.set_size_inches(4, 4)
    if name:
        fig.savefig(name)

def plot_models(df, column, norm=False, **kwargs):
    avg_pivot = df.pivot_table(index='model', values=[column]).sort_values(column, ascending=False)
    c_pivot = df.pivot(index='quality', columns='model', values=column).reindex(columns=avg_pivot.index)
    title = f'{column}: all models'
    if norm:
        c_pivot = c_pivot / c_pivot.max(axis=0)
        title += ' (scaled)'
    ax = c_pivot.plot(**kwargs)
    ax.set_title(title)
    return ax


In [9]:
ax = plot_models(c_df, 'AP', ylim=None)
# _save_plot(ax, 'all_AP_raw.pdf')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/02-all_AP_raw")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/02-all_AP_raw.png and 2021-03-01-Plot_reval_05_APs/02-all_AP_raw.pdf


In [10]:
ax = plot_models(c_df, 'AP', True)
_save_plot(ax, 'all_AP_scaled.pdf')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/03-all_AP_scaled")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/03-all_AP_scaled.png and 2021-03-01-Plot_reval_05_APs/03-all_AP_scaled.pdf


In [11]:
for mc in c_df.model.unique():
    m_df = c_df[c_df.model == mc][['quality', 'APl', 'APm', 'APs']]
    ax = m_df.plot(x='quality', ylim=YLIM_APL, color=COLOR_SML)
    ax.set_title(mc)
    save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/04-APsml-{mc}")
    plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/04-APsml-R101.png and 2021-03-01-Plot_reval_05_APs/04-APsml-R101.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/04-APsml-R101_C4.png and 2021-03-01-Plot_reval_05_APs/04-APsml-R101_C4.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/04-APsml-R101_DC5.png and 2021-03-01-Plot_reval_05_APs/04-APsml-R101_DC5.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/04-APsml-R101_FPN.png and 2021-03-01-Plot_reval_05_APs/04-APsml-R101_FPN.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/04-APsml-R50.png and 2021-03-01-Plot_reval_05_APs/04-APsml-R50.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/04-APsml-R50_C4.png and 2021-03-01-Plot_reval_05_APs/04-APsml-R50_C4.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/04-APsml-R50_DC5.png and 2021-03-01-Plot_reval_05_APs/04-APsml-R50_DC5.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/04-APsml-R50_FPN.png and 2021-03-01-Plot_reval_05_APs/04-APsml-R50_FPN.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/04-APsm

In [12]:
# no color=COLOR_SML -- this plot is too bad anyway
pd.pivot(c_df, index='quality', columns='model', values=['APs', 'APm', 'APl']).plot(ylim=YLIM_APL);
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/05-APsml-all")
plt.close();

Figure saved to 2021-03-01-Plot_reval_05_APs/05-APsml-all.png and 2021-03-01-Plot_reval_05_APs/05-APsml-all.pdf


In [13]:
mAPsml = c_df.groupby('quality').agg({x: 'mean' for x in reversed(('APs', 'APm', 'APl'))})
mAPsml = mAPsml / mAPsml.max(axis=0)
plot = mAPsml.plot(color=COLOR_SML)
_save_plot(plot, 'average_APlms_scaled.pdf', 'Per-size AP (scaled)')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/06-APsml-allavg-scaled")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/06-APsml-allavg-scaled.png and 2021-03-01-Plot_reval_05_APs/06-APsml-allavg-scaled.pdf


In [14]:
plot = c_df.groupby('quality').agg({x: 'mean' for x in reversed(('APs', 'APm', 'APl'))}).plot(ylim=YLIM_AP, color=COLOR_SML)
_save_plot(plot, 'average_APlms.pdf', "Per-size AP (large, medium, small)")
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/07-APsml-allavg-raw")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/07-APsml-allavg-raw.png and 2021-03-01-Plot_reval_05_APs/07-APsml-allavg-raw.pdf


# "Rich" results, with TPR, PPV and F1
Computed for Tc = 0.5, used to have wrong results (crowds, i.e. EX, counted as TP)

In [15]:
rich_files = glob.glob(f'{REVAL_DIR}/*/*/rich_results.json')

df = pd.DataFrame(load(f) for f in rich_files).sort_values(['model', 'quality'])
df.tail()

,quality,model,elapsed,tp,fp,ex,precision,recall,f1,min_score,score_T,AP,AP50,AP75,APs,APm,APl
133,97,X101,2950.533190,29538,69427,11885,0.298469,0.812935,0.436630,0.050000,None,43.125026,63.823199,47.014898,27.212577,46.190293,54.863181
175,98,X101,2949.976151,29537,69465,11845,0.298348,0.812908,0.436496,0.050000,None,43.064660,63.628361,46.943094,26.896694,46.098858,54.857996
103,99,X101,2952.297939,29549,69465,11835,0.298433,0.813238,0.436634,0.050000,None,43.101607,63.681584,47.133904,27.035522,46.123877,54.916508
112,100,X101,2952.063746,29549,69454,11853,0.298466,0.813238,0.436670,0.050001,None,43.070649,63.715295,46.980151,26.992763,46.103360,54.878414
406,101,X101,2673.639724,29546,69608,11862,0.297981,0.813155,0.436139,0.050000,None,43.046948,63.651725,46.888371,27.205004,46.093387,54.890155


In [16]:
for mc in df.model.unique():
    ax = df[df.model == mc][['quality', 'precision', 'recall', 'f1']].plot(x='quality', ylim=YLIM_F1)
    ax.set_title(mc)
    _save_plot(ax, f'{mc}_prf1.pdf')
    save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/08-PRF1-{mc}")
    plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/08-PRF1-R101.png and 2021-03-01-Plot_reval_05_APs/08-PRF1-R101.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/08-PRF1-R101_C4.png and 2021-03-01-Plot_reval_05_APs/08-PRF1-R101_C4.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/08-PRF1-R101_DC5.png and 2021-03-01-Plot_reval_05_APs/08-PRF1-R101_DC5.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/08-PRF1-R101_FPN.png and 2021-03-01-Plot_reval_05_APs/08-PRF1-R101_FPN.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/08-PRF1-R50.png and 2021-03-01-Plot_reval_05_APs/08-PRF1-R50.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/08-PRF1-R50_C4.png and 2021-03-01-Plot_reval_05_APs/08-PRF1-R50_C4.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/08-PRF1-R50_DC5.png and 2021-03-01-Plot_reval_05_APs/08-PRF1-R50_DC5.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/08-PRF1-R50_FPN.png and 2021-03-01-Plot_reval_05_APs/08-PRF1-R50_FPN.pdf
Figure saved to 2021-03-01-Plot_reval_05_APs/08-PRF1-X101.png and 20

In [17]:
ax = plot_models(df, 'precision', ylim=None)
_save_plot(ax, 'all_PPV_raw.pdf')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/09-all_PPV_raw")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/09-all_PPV_raw.png and 2021-03-01-Plot_reval_05_APs/09-all_PPV_raw.pdf


In [18]:
ax = plot_models(df, 'recall', ylim=YLIM_F1)
_save_plot(ax, 'all_TPR_raw.pdf')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/10-all_TPR_raw")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/10-all_TPR_raw.png and 2021-03-01-Plot_reval_05_APs/10-all_TPR_raw.pdf


In [19]:
ax = plot_models(df, 'f1') # ylim=YLIM_F1
_save_plot(ax, 'all_F1_raw.pdf')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/11-all_F1_raw")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/11-all_F1_raw.png and 2021-03-01-Plot_reval_05_APs/11-all_F1_raw.pdf


# Structural similarity (SSIM)

In [20]:
qdf = pd.read_csv('../data/val2017_means_per_Q.csv')
qdf.head(2)

,quality,mean_squared_error,peak_signal_noise_ratio,structural_similarity,filesize,B_per_kpx
0,1,536.18,21.205,0.539,7203.782,9.291
1,2,535.91,21.207,0.539,7207.225,9.295


In [21]:
avgdf = df.groupby('quality').agg('mean')
avgdf.tail(2)

,elapsed,tp,fp,ex,precision,recall,f1,min_score,AP,AP50,AP75,APs,APm,APl
quality,,,,,,,,,,,,,,
100,1144.756110,29614.777778,164210.111111,13742.888889,0.183927,0.815048,0.294050,0.050001,40.381796,60.816121,43.674192,23.457767,44.269027,53.271776
101,1097.117104,29613.444444,164403.333333,13744.888889,0.183683,0.815012,0.293742,0.050001,40.392295,60.849938,43.650912,23.476854,44.283071,53.287806


In [22]:
rich = pd.merge(avgdf, qdf, left_index=True, right_index=True)

In [23]:
ax = rich.plot(x='structural_similarity', y='recall', xlim=(0, 1), ylim=(0, 1))
_save_plot(ax, 'recall_by_ssim.pdf')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/12-recall_by_ssim")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/12-recall_by_ssim.png and 2021-03-01-Plot_reval_05_APs/12-recall_by_ssim.pdf


In [24]:
ax = rich.plot(x='structural_similarity', y='precision')#, xlim=(0, 1), ylim=(0, 1))
_save_plot(ax, '')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/13-precision_by_ssim")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/13-precision_by_ssim.png and 2021-03-01-Plot_reval_05_APs/13-precision_by_ssim.pdf


In [25]:
ax = rich.plot(x='structural_similarity', y='f1')#, xlim=(0, 1), ylim=(0, 1))
_save_plot(ax, '')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/14-F1_by_ssim")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/14-F1_by_ssim.png and 2021-03-01-Plot_reval_05_APs/14-F1_by_ssim.pdf


In [26]:
ax = rich.plot(x='quality', y='structural_similarity', ylim=(0, 1))
# , xlabel="jakość JPEG (Q)", ylabel="wartość SSIM", legend=False
_save_plot(ax, 'SSIM_by_q.pdf')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/15-SSIM_by_quality")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/15-SSIM_by_quality.png and 2021-03-01-Plot_reval_05_APs/15-SSIM_by_quality.pdf


In [27]:
ax = rich[['structural_similarity', 'precision', 'recall']].plot(x='structural_similarity', xlim=(0, 1), ylim=(0, 1))
_save_plot(ax, 'pr_by_SSIM.pdf')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/16-pr_by_SSIM")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/16-pr_by_SSIM.png and 2021-03-01-Plot_reval_05_APs/16-pr_by_SSIM.pdf


In [28]:
ax = rich.plot(x='quality', y='AP', ylim=(0, 100), xlim=(0, 100))
_save_plot(ax, '')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/17-avgAP_by_quality")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/17-avgAP_by_quality.png and 2021-03-01-Plot_reval_05_APs/17-avgAP_by_quality.pdf


In [29]:
ax = rich.plot(x='structural_similarity', y='AP', xlim=(0, 1), ylim=(0, 100))
_save_plot(ax, 'AP_by_SSIM.pdf')
save_plot(plt.gcf(), f"2021-03-01-Plot_reval_05_APs/18-avgAP_by_SSIM")
plt.close()

Figure saved to 2021-03-01-Plot_reval_05_APs/18-avgAP_by_SSIM.png and 2021-03-01-Plot_reval_05_APs/18-avgAP_by_SSIM.pdf


# Tables

In [30]:
rich.loc[[96, 90, 80, 60, 40, 20], ['AP', 'APl', 'APm', 'APs']].round(1).reset_index()

,index,AP,APl,APm,APs
0,96,40.3,53.2,44.2,23.5
1,90,40.1,53.1,43.9,23.3
2,80,39.4,52.5,43.0,22.1
3,60,38.0,51.3,41.7,20.0
4,40,36.1,49.5,39.6,18.2
5,20,28.4,40.1,31.1,13.1


In [31]:
rich['PPV'] = rich.precision * 100
rich['TPR'] = rich.recall * 100
rich['SSIM'] = rich.structural_similarity * 100
rich['size'] = rich.filesize / 1024
print(rich
     .replace('precision', 100 * rich.precision)
     .set_index('quality')
     .loc[
         [96, 90, 80, 60, 40, 20, 10],
         ['AP', 'AP50', 'AP75', 'APl', 'APm', 'APs', 'TPR', 'PPV', 'SSIM', 'size']
     ]
     #.reset_index()
     .round(1)
     .to_latex()
)

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &    AP &  AP50 &  AP75 &   APl &   APm &   APs &   TPR &   PPV &  SSIM &   size \\
quality &       &       &       &       &       &       &       &       &       &        \\
\midrule
96      &  40.3 &  60.7 &  43.5 &  53.2 &  44.2 &  23.5 &  81.4 &  18.4 &  99.9 &  167.1 \\
90      &  40.1 &  60.5 &  43.2 &  53.1 &  43.8 &  23.2 &  81.2 &  18.5 &  97.1 &  107.0 \\
80      &  39.4 &  59.6 &  42.4 &  52.5 &  43.0 &  22.2 &  80.5 &  18.5 &  93.4 &   74.6 \\
60      &  37.9 &  57.8 &  40.8 &  51.3 &  41.5 &  20.1 &  79.1 &  18.1 &  89.5 &   46.5 \\
40      &  36.0 &  55.5 &  38.4 &  49.4 &  39.6 &  18.3 &  77.5 &  17.3 &  86.3 &   33.9 \\
20      &  27.3 &  44.2 &  28.5 &  38.5 &  30.2 &  12.6 &  69.7 &  13.8 &  80.5 &   21.3 \\
10      &  10.8 &  18.7 &  10.8 &  15.9 &  12.3 &   4.8 &  43.0 &   8.0 &  72.6 &   13.5 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_31567/700065347.py:14: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  .to_latex()
